# Document Question-Answer [h2oGPT](https://github.com/h2oai/h2ogpt)

In this notebook, we demonstrate how one can use h2oGPT with a large language model.

To begin, please get free ngrok account to get auth token (e.g.) using your Google email/login and get token: https://dashboard.ngrok.com/get-started/setup .  You will be asked for this token below in an input box.

In [1]:
!git clone https://github.com/h2oai/h2ogpt.git
!cd h2ogpt && git checkout 2668694581347b0d1afe76760213db46f7214126 -q
!cp -ar h2ogpt/. ./
!rm -r h2ogpt

Cloning into 'h2ogpt'...
remote: Enumerating objects: 9204, done.
remote: Counting objects: 100% (1703/1703), done.
remote: Compressing objects: 100% (375/375), done.
remote: Total 9204 (delta 1413), reused 1496 (delta 1322), pack-reused 7501
Receiving objects: 100% (9204/9204), 16.93 MiB | 6.41 MiB/s, done.
Resolving deltas: 100% (6161/6161), done.


In [2]:
# Install pyhon 3.10 that will be used within pipenv
!sudo add-apt-repository ppa:deadsnakes/ppa -y > /dev/null
!sudo apt install python3.10 python3.10-distutils psmisc -y > /dev/null
!curl -sS https://bootstrap.pypa.io/get-pip.py | python3.10 > /dev/null

In [3]:
# Install dependencies
!for fil in requirements.txt reqs_optional/requirements_optional_langchain.txt reqs_optional/requirements_optional_gpt4all.txt reqs_optional/requirements_optional_langchain.gpllike.txt reqs_optional/requirements_optional_langchain.urls.txt ; do pip install -r $fil ; done


  Cloning https://github.com/huggingface/peft.git (to revision 0b62b4378b4ce9367932c73540349da9a41bdea8) to /tmp/pip-req-build-13nzqnb6
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-13nzqnb6
  Running command git rev-parse -q --verify 'sha^0b62b4378b4ce9367932c73540349da9a41bdea8'
  Running command git fetch -q https://github.com/huggingface/peft.git 0b62b4378b4ce9367932c73540349da9a41bdea8
  Running command git checkout -q 0b62b4378b4ce9367932c73540349da9a41bdea8
  Resolved https://github.com/huggingface/peft.git to commit 0b62b4378b4ce9367932c73540349da9a41bdea8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Ignoring pypandoc: markers 'sys_platform == "darwin" and platform_machine == "arm64"' don't match your environment
  Using cached numpy-1.24.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Using cached

In [4]:
# download llama model if running that:
!rm -rf WizardLM-7B-uncensored.ggmlv3.q8_0.bin*
!wget https://huggingface.co/TheBloke/WizardLM-7B-uncensored-GGML/resolve/main/WizardLM-7B-uncensored.ggmlv3.q8_0.bin

--2023-07-10 23:45:30--  https://huggingface.co/TheBloke/WizardLM-7B-uncensored-GGML/resolve/main/WizardLM-7B-uncensored.ggmlv3.q8_0.bin
Resolving huggingface.co (huggingface.co)... 65.8.178.118, 65.8.178.27, 65.8.178.12, ...
Connecting to huggingface.co (huggingface.co)|65.8.178.118|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/c0/cd/c0cd768b4cd58780ae60ca18240a853723360aac1874854c9e07bc87d943ee47/2802e2c7ffb3cae9bab40425a2600d286b98ed5cc7fe2116fc205a2a101e913e?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27WizardLM-7B-uncensored.ggmlv3.q8_0.bin%3B+filename%3D%22WizardLM-7B-uncensored.ggmlv3.q8_0.bin%22%3B&response-content-type=application%2Foctet-stream&Expires=1689291930&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY4OTI5MTkzMH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9jMC9jZC9jMGNkNzY4YjRjZDU4NzgwYWU2MGNhMTgyNDBhODUzNzIzMzYwY

In [5]:
# Sign-up for free ngrok account using (e.g.) your Google email/login and get token: https://dashboard.ngrok.com/get-started/setup

!pip install pyngrok
import getpass
from pyngrok import ngrok, conf

print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
conf.get_default().auth_token = getpass.getpass()

# Open an http ngrok tunnel
connection_string = ngrok.connect(7860, "http").public_url
print("Go to this address in about 20 seconds, and click on Visit Site: %s" % connection_string)

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth
··········


Go to this address in about 20 seconds, and click on Visit Site: https://77c2-35-243-196-63.ngrok-free.app


In [6]:
get_ipython().system_raw("""GRADIO_SERVER_PORT=7860 python generate.py --base_model='llama' --prompt_type=wizard2 --score_model=None --langchain_mode=LLM --visible_langchain_modes="['LLM', 'UserData', 'MyData']" --user_path=user_path --share=False &> logs.txt &""")
# wait a bit for server to come up
import time
time.sleep(20)

In [7]:
# can see what is running:
# !ps -auxwf

In [8]:
# can check logs
# !cat logs.txt

In [9]:
# can kill old ngrok + generate and try again, or just restart entire runtime + run all cells
do_kill = False
if do_kill:
  !pkill -f generate --signal 9
  !pkill -f frpc_linux_amd --signal 9
  !pkill -f ngrok --signal 9